# Notebook for Manual MCS Setup
- Note: NAMESPACE and target_board variables must be updated before proceeding with your work.

In [2]:
import os
import json
from time import sleep
import tango
from PyTango import Database, DeviceProxy

print("Setting up parameters\n")

NAMESPACE = "ci-ska-mid-cbf-system-tests-1349266288-ackyzrxu"

TANGO_HOST = f"databaseds-tango-base.{NAMESPACE}.svc.cluster.local:10000"
os.environ["TANGO_HOST"] = TANGO_HOST

# Config files set up
DATA_DIR = "../../data"

# MCS/Talon setup
target_board = [1,2,3,4,5,6,7,8]
COMMON_CONFIG_FOLDER = os.path.join(DATA_DIR, "mid_telescope/cbf")
HW_CONFIG_FOLDER = os.path.join(COMMON_CONFIG_FOLDER, "hw_config")
INIT_SYS_PARAM_FILE = os.path.join(COMMON_CONFIG_FOLDER, "sys_params/initial_system_param.json")
TARGET_MAC_ADDRESS = "08:c0:eb:9d:47:78"
url = "https://142.73.34.170/" + NAMESPACE + "/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run"
print(
    "You can monitor board status using: https://142.73.34.170/{}/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run".format(
        NAMESPACE
    )
)

Setting up parameters

You can monitor board status using: https://142.73.34.170/ci-ska-mid-cbf-system-tests-1349266288-ackyzrxu/taranta/dashboard?id=65e7b6f7b72ec70018cdb16a&mode=run


### MCS Deployer Setup

In [ ]:
db = Database()

deployer_tango = DeviceProxy("mid_csp_cbf/ec/deployer")
# Check the devices initially deployed to the database
print(*db.get_device_exported("*").value_string, sep="\n")
# Make sure the deployer device is set to ON
deployer_tango.On()
print(deployer_tango.state())

if any(i > 4 for i in target_board):
    print("Using swap for higher number talons")
    config = "hw_config_swap_psi.yaml"
    print("Modifying target to use lower nums to match swap file")
    target_board = list(map(lambda x: x - 4, target_board))
else:
    print("Using standard HW config")
    config = "hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(HW_CONFIG_FOLDER, config)
if os.path.isfile(HW_CONFIG_FILE):
    print("HW config: ✔️")
else:
    print("hw config: ❌")

In [ ]:
!kubectl cp $HW_CONFIG_FILE $NAMESPACE/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml

In [ ]:
deployer_tango.targetTalons = target_board
print(deployer_tango.targetTalons)

In [ ]:
deployer_tango.generate_config_jsons()

In [ ]:
deployer_tango.set_timeout_millis(200000)
try:
    deployer_tango.download_artifacts()
except tango.DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer_tango.set_timeout_millis(4500)

In [ ]:
deployer_tango.configure_db()

## Turning MCS On

In [ ]:
control_tango = DeviceProxy("mid-csp/control/0")
print(control_tango.status())
subarray_tango = DeviceProxy("mid-csp/subarray/01")
print(subarray_tango.status())
CBF_tango = DeviceProxy("mid_csp_cbf/sub_elt/controller")
print(CBF_tango.status())

In [ ]:
# Set relevant values on the mid-csp controller
control_tango.adminMode = 0
sleep(1)
control_tango.write_attribute("cbfSimulationMode", 0)
control_tango.cbfSimulationMode = 0

if (
    control_tango.read_attribute("adminMode").value == 0
    and control_tango.read_attribute("cbfSimulationMode").value == 0
):
    print("Set values successfully!")
elif (
    control_tango.read_attribute("adminMode").value == 0
    and control_tango.read_attribute("cbfSimulationMode").value == 1
):
    print("Set to simulation mode on.")
else:
    print("Error, couldn't set values!")

In [ ]:
with open(INIT_SYS_PARAM_FILE, encoding="utf-8") as init_file:
    data = json.load(init_file)
print(json.dumps(data))

In [ ]:
upload_result = CBF_tango.InitSysParam(json.dumps(data))
print(upload_result[1])
print(CBF_tango.sysParam)

In [ ]:
print(control_tango.status())
control_tango.set_timeout_millis(50000)
target = ["mid_csp_cbf/sub_elt/controller"]
control_tango.On(target)
sleep(55)
print(control_tango.status())

In [ ]:
print(control_tango.status())

## Bite

In [ ]:
bite_tango = DeviceProxy("mid_csp_cbf/ec/bite")
# Running this should return RUNNING
print(bite_tango.State())

In [ ]:
bite_tango.boards = target_board
bite_tango.bite_mac_address = TARGET_MAC_ADDRESS

In [ ]:
standin_board = target_board[0]
print(standin_board)
!kubectl exec -ti -n $NAMESPACE ec-bite -- python3 midcbf_bite.py --talon-bite-config --boards=$standin_board --bite_mac_address=$TARGET_MAC_ADDRESS

In [ ]:
!kubectl -n $NAMESPACE logs ds-bite-bite-0 | grep generate_bite_data

In [ ]:
bite_tango.start_lstv_replay()

In [ ]:
!kubectl -n $NAMESPACE logs ds-bite-bite-0 | grep start_lstv_replay